In [1]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
# 讀取訓練集和測試集
X_train = pd.read_csv('X_train.csv')
X_test = pd.read_csv('X_test.csv')
y_train = pd.read_csv('y_train.csv')['country_destination']
y_test = y_train[:len(X_test)]

In [3]:
# 設定要調整的參數和範圍
# params = {
#     'learning_rate': [0.1,0.3],
#     'max_depth': [6,7,8,9,10],
#     'subsample': [0.5],
#     'colsample_bytree': [0.5],
#     'n_estimators': [25,50]
# }

# # 建立 XGBClassifier 模型
# xgb_model  = XGBClassifier(tree_method='gpu_hist')

# label_encoder = LabelEncoder()
# y_train_encoded = label_encoder.fit_transform(y_train)

# # 使用 GridSearchCV 進行參數調整
# # 使用 GridSearchCV 進行參數調整
# grid_search = GridSearchCV(xgb_model, param_grid=params, cv=5, scoring='accuracy')
# grid_search.fit(X_train, 
#                 y_train_encoded, 
#                 early_stopping_rounds=10, 
#                 eval_metric='mlogloss', 
#                 eval_set=[(X_test, label_encoder.transform(y_test))])


# # 輸出最佳參數
# print("Best parameters found: ", grid_search.best_params_)

# # 使用最佳參數重新建立模型並預測
# best_model = XGBClassifier(**grid_search.best_params_)
# best_model.fit(X_train, y_train_encoded)
# pred = best_model.predict(X_test)
# xgb_pred = label_encoder.inverse_transform(pred)
# print(xgb_pred.shape)


In [6]:
# 建立 XGBClassifier 模型
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

xgb = XGBClassifier(max_depth=6, 
                    learning_rate=0.3, 
                    n_estimators=25,
                    objective='multi:softprob', 
                    subsample=0.5, 
                    colsample_bytree=0.5,
                    seed=0,
                    tree_method='gpu_hist')

xgb.fit(X_train, y_train_encoded)
pred = xgb.predict(X_test)  
xgb_pred = label_encoder.inverse_transform(pred)
print(xgb_pred.shape)

(62096,)


In [7]:

def metrics_sklearn(y_valid, y_pred):
    accuracy = accuracy_score(y_valid, y_pred)
    print('AccuracyL %.2f%%' % (accuracy * 100))

    precision = precision_score(y_valid, y_pred, average='macro')
    print('Precision: %.2f%%' % (precision *100))

    recall = recall_score(y_valid, y_pred, average='macro')
    print('Recall: %.2f %%' % (recall *100))

    f1 = f1_score(y_valid, y_pred, average='macro')
    print('F1: %.2f%%' % (f1*100))
    

In [8]:
metrics_sklearn(y_test, xgb_pred)

AccuracyL 49.52%


ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].